In [84]:
import xml
import xml.etree.ElementTree as ET
import os
from os import path
import glob
from collections import defaultdict
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Setting up path related information

In [85]:
base_data_dir = "/home/shtoshni/Research/events/data/kbp_2014-2015/data/2015"
output_dir = "/home/shtoshni/Research/events/data/kbp_2014-2015/"

split_to_subdir_dict = {
    "training": {"source": "source", "mention": "event_nugget", "coref": "event_hopper"},
    "eval": {"source": "source", "mention": "nugget", "coref": "hopper"}
}

file_suffix = {"mention": ".event_nuggets.xml", "coref": ".event_hoppers.xml"}

### Set split to "training"

In [86]:
SPLIT = 'training'
split_dir = path.join(base_data_dir, SPLIT)
subdir_dict = split_to_subdir_dict[SPLIT]

### Mention stats

In [87]:
# Get all mention files
mention_dir = path.join(split_dir, subdir_dict["mention"])
mention_ann_files = sorted(glob.glob(path.join(mention_dir, "*.xml")))

# Global counters
doc_types = defaultdict(int)
event_type_count = defaultdict(int)
event_subtype_count = defaultdict(int)
event_realis_count = defaultdict(int)
trigger_word_dict = defaultdict(int)
span_to_count = defaultdict(set)

for ann_file in mention_ann_files:
    tree = ET.parse(ann_file)
    xml_root = tree.getroot()

    doc_id = xml_root.attrib['doc_id']
    doc_source_type = xml_root.attrib['source_type']
    
    # Update doc type count
    doc_types[doc_source_type] += 1

    for elem in xml_root.iter('event_mention'):
        event_id = elem.attrib['id']
        event_type = elem.attrib['type']
        # Update event type count
        event_type_count[event_type] += 1
        
        event_subtype = f"{event_type}.{elem.attrib['subtype']}"
        # Update event subtype count
        event_subtype_count[event_subtype] += 1
        
        event_realis = elem.attrib['realis']
        event_realis_count[event_realis] += 1
        
        trigger_elem = elem.find('trigger')
        offset = trigger_elem.attrib['offset']
        length = trigger_elem.attrib['length']
        event_phrase = trigger_elem.text
        trigger_word_dict[event_phrase] += 1
        
        # Doc ID, doc offset, and span length uniquely identify a span 
        span_to_count[(doc_id, offset, length)].add(event_id)
        
        
trigger_count_per_span = defaultdict(int)
for (doc_id, offset, length), event_id_list in span_to_count.items():
    trigger_count_per_span[len(event_id_list)] += 1

### Plot mention stats

In [88]:
fig = make_subplots(rows=2, cols=2, 
                   subplot_titles=("Document Type", "Event Type", "Realis Type", "Event per Span"))


x, y = zip(*sorted(doc_types.items()))
fig.add_trace(
    go.Bar(x=x, y=y, width=0.2),
    row=1, col=1
)

x, y = zip(*sorted(event_type_count.items()))
fig.add_trace(
    go.Bar(x=x, y=y, width=0.2),
    row=1, col=2
)

x, y = zip(*sorted(event_realis_count.items()))
fig.add_trace(
    go.Bar(x=x, y=y, width=0.2),
    row=2, col=1
)

x, y = zip(*sorted(trigger_count_per_span.items()))
x = [str(x_elem) for x_elem in x]  # The X-labels need to be a string
fig.add_trace(
    go.Bar(x=x, y=y, width=0.2),
    row=2, col=2
)

fig.update_layout(showlegend=False)
fig.show()


### Other Mention Stats

In [89]:
K = 10

# Subtype counts
subtype_counts = sorted(event_subtype_count.items(), key=lambda x: x[1], reverse=True)
print(f'Top-{K} event subtypes\n')
for subtype, count in subtype_counts[:K]:
    print(subtype, count)

print('\n')

# Trigger words
trigger_word_counts = sorted(trigger_word_dict.items(), key=lambda x: x[1], reverse=True)
print(f'Top-{K} event triggers\n')
for trigger_word, count in trigger_word_counts[:K]:
    print(trigger_word, count)


Top-10 event subtypes

conflict.attack 800
transaction.transfermoney 551
movement.transportperson 517
life.die 514
contact.broadcast 417
contact.contact 337
justice.arrestjail 287
transaction.transferownership 280
contact.meet 244
justice.pardon 239


Top-10 event triggers

said 194
convicted 125
pardon 94
told 83
war 79
kill 78
pardoned 73
killed 72
death 70
trial 63


### Coref Stats

In [90]:
# Get all mention files
coref_dir = path.join(split_dir, subdir_dict["coref"])
coref_ann_files = sorted(glob.glob(path.join(coref_dir, "*.xml")))

# Global counters
event_type_counter = defaultdict(int)
event_subtype_counter = defaultdict(int)
event_realis_counter = defaultdict(int)
event_phrase_counter = defaultdict(int)
coref_chain_size_counter = defaultdict(int)

for ann_file in coref_ann_files:
    tree = ET.parse(ann_file)
    xml_root = tree.getroot()
    
    for coref_elem in xml_root.iter('hopper'):
        event_type_set = set()
        event_subtype_set = set()
        event_realis_set = set()
        event_phrase_set = set()
        coref_chain_size = 0
        for event_elem in coref_elem.iter('event_mention'):
            # Get type, subtype, realis, and text
            event_type_set.add(event_elem.attrib['type'])
            event_subtype_set.add(f"{event_type}.{event_elem.attrib['subtype']}")
            event_realis_set.add(event_elem.attrib['realis'])

            trigger_elem = event_elem.find('trigger')
            event_phrase_set.add(trigger_elem.text)
            coref_chain_size += 1
            
        event_type_counter[len(event_type_set)] += 1
        event_subtype_counter[len(event_subtype_set)] += 1
        event_realis_counter[len(event_realis_set)] += 1
        event_phrase_counter[len(event_phrase_set)] += 1
        coref_chain_size_counter[coref_chain_size] += 1

### Plot Coref Chain Stats

In [91]:
fig = make_subplots(rows=3, cols=2, 
                   subplot_titles=("Hist. of Different Event Types in a Coref Chain", 
                                   "Hist. of Different Event Subtypes in a Coref Chain", 
                                   "Hist. of Different Realis Types in a Coref Chain", 
                                   "Hist. of Different Mention Phrases in a Coref Chain",
                                   "Hist. of Coref Chain Size",
                                  )
                   )


x, y = zip(*sorted(event_type_counter.items()))
fig.add_trace(
    go.Bar(x=x, y=y, width=0.2),
    row=1, col=1
)

x, y = zip(*sorted(event_subtype_counter.items()))
fig.add_trace(
    go.Bar(x=x, y=y, width=0.2),
    row=1, col=2
)

x, y = zip(*sorted(event_realis_counter.items()))
fig.add_trace(
    go.Bar(x=x, y=y, width=0.2),
    row=2, col=1
)

x, y = zip(*sorted(event_phrase_counter.items()))
fig.add_trace(
    go.Bar(x=x, y=y, width=0.2),
    row=2, col=2
)
                    
x, y = zip(*sorted(coref_chain_size_counter.items()))
fig.add_trace(
    go.Bar(x=x, y=y, width=0.2),
    row=3, col=1
)                   

fig.update_layout(showlegend=False)
fig.show()